In [ ]:
import os 
import importlib

from waffles.input_output.hdf5_structured import load_structured_waveformset
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.Waveform import Waveform
from waffles.np04_data.ProtoDUNE_HD_APA_maps import APA_map # apa map
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid # channel grid

import integral_computation_function as integral_module
from integral_computation_function import channel_integral_computation

import waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots as utils_plots_module
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *


from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak import MyAnaPeak

import waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak_NEW as MyAnaPeak_NEW_module


In [ ]:
# Reading full streaming data

run_folder = "/afs/cern.ch/work/a/anbalbon/public/reading_rucio/run027361/full_streaming/no_selection"
fname = "processed_np04hd_raw_run027361_0001_dataflow0_datawriter_0_20240622T004415.hdf5_structured.hdf5"
filepath = os.path.join(run_folder, fname)
FS_wfset = load_structured_waveformset(filepath)  # load HDF5 structured waveform set  

In [ ]:
# Plotting original full-streaming waveforms --> Negative signals

importlib.reload(utils_plots_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *
plotting_overlap_wf_PEAK_NEW(FS_wfset, n_wf= 20, show  = True, peak_bool = False, peak_beam_bool = False)

In [ ]:
# Peak finding analysis for beam event selection 
importlib.reload(MyAnaPeak_NEW_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak_NEW import MyAnaPeak_NEW

analysis_label = 'peak_finding'
analysis_class= MyAnaPeak_NEW

ip = IPDict(baseline_limits= [0, 200],
            n_std = 5,
            peak_distance= 500,
            beam_timeoffset_limits = [-40, -20],
            signal_sign = "negative"
            )

_ = FS_wfset.analyse(label=analysis_label ,analysis_class=analysis_class, input_parameters=ip, checks_kwargs = IPDict({'points_no': FS_wfset.points_per_wf}), overwrite=True)

print('done\n\n')

In [ ]:
# Plotting full-streaming waveforms with peak finding results --> Negative signals

importlib.reload(utils_plots_module) 
from waffles.np04_analysis.lightyield_vs_energy.scripts.utils_plots import *
plotting_overlap_wf_PEAK_NEW(FS_wfset, n_wf= 20, show  = True, peak_bool = False, peak_beam_bool = True, analysis_label = analysis_label)

In [ ]:
# Creo nuovo wfset per fare prove sulla selezione della finistra di full-streaming

# Lo slice adcs funziona per selezionare gli adcs, ma non trasla le altre informazioni 
# Tutte le info della wf non vengono modificare (timestamp, daq_window_timestamp, run_number,record_number,endpoint,channel,trigger_type)
# ad eccezione dello starting_tick che di default è 0 di solito
# Per quanto riguarda il wfset dopo che ho fatto apply, gli attributi rimango uguali compreso points_per_wf 187520


# Da fare: trovare 1/3 dell'ampiezza del picco e selezionare la finestra intorno 
# Da valutare: conviene rimuovere da prima la baseline? probabilmente si (magari usa metodo julio)

In [ ]:
prova_wfset = load_structured_waveformset(filepath)

In [ ]:
plotting_overlap_wf_PEAK_NEW(prova_wfset, n_wf= 2, show  = True, peak_bool = False, peak_beam_bool = False, analysis_label = analysis_label)

In [ ]:
# Function to remove baseline and reverse the signal (if needed --> good for find_peaks)

from waffles.data_classes.WaveformAdcs import WaveformAdcs
from waffles.Exceptions import GenerateExceptionMessage

def my_subtract_baseline(
        waveform: WaveformAdcs,
        baseline_analysis_label: str,
        inversion: bool
) -> None:
    """This method overwrites the adcs method of the given
    WaveformAdcs object, by subtracting its baseline.

    waveform: WaveformAdcs
        The waveform whose adcs will be modified
    baseline_analysis_label: str
        The baseline to subtract must be available 
        under the 'baseline' key of the result of the analysis
        whose label is given by this parameter, i.e. in
        waveform.analyses[analysis_label].result['baseline']
    inversion: bool
        If True, the baseline-subtracted signal is inverted.
        If False, the baseline-subtracted signal is not inverted.
    """

    try:
        baseline = waveform.analyses[baseline_analysis_label].result['baseline']

    except KeyError:
        raise Exception(
            GenerateExceptionMessage(
                1,
                "subtract_baseline()",
                f"The given waveform does not have the analysis"
                f" '{baseline_analysis_label}' in its analyses "
                "attribute, or it does, but the 'baseline' key "
                "is not present in its result."
            )
        )
    
    if inversion:
        waveform._WaveformAdcs__set_adcs(
            baseline - waveform.adcs
        )
    else:
        waveform._WaveformAdcs__set_adcs(
            waveform.adcs - baseline
        )

    return

In [ ]:
#Tolgo la baseline subito e inverto 

from waffles.np04_analysis.led_calibration.utils import compute_average_baseline_std #baseline computation
from waffles.utils.baseline.baseline_utils import subtract_baseline # baseline subtraction
from waffles.utils.baseline.WindowBaseliner import WindowBaseliner
from waffles.data_classes.StoreWfAna import StoreWfAna 


baseline_limits = [0, 200]
baseliner_std_cut = 3.
baseliner_type = 'mean'
baseline_analysis_label = 'baseliner'
null_baseline_analysis_label = 'null_baseliner'


# Baseline
baseliner_input_parameters = IPDict({'baseline_limits': baseline_limits, 'std_cut': baseliner_std_cut, 'type': baseliner_type})    
checks_kwargs = IPDict({'points_no': prova_wfset.points_per_wf})
_ = prova_wfset.analyse(baseline_analysis_label, WindowBaseliner, baseliner_input_parameters, checks_kwargs=checks_kwargs, overwrite=True)

# Compute average baseline std
average_baseline_std = compute_average_baseline_std(prova_wfset, baseline_analysis_label)

# Remove baseline
prova_wfset.apply(my_subtract_baseline, baseline_analysis_label, inversion = True, show_progress=False)

In [ ]:
plotting_overlap_wf_PEAK_NEW(prova_wfset, n_wf= 5, show  = True, peak_bool = False, peak_beam_bool = False, analysis_label = analysis_label)

In [ ]:
# Goal: search fpr peak 
importlib.reload(MyAnaPeak_NEW_module)
from waffles.np04_analysis.lightyield_vs_energy.scripts.MyAnaPeak_NEW import MyAnaPeak_NEW
analysis_class = MyAnaPeak_NEW
analysis_label = 'finding_peaks'

ip = IPDict(baseline_limits= [0, 200],
            n_std = 5,
            peak_distance= 500,
            beam_timeoffset_limits = [-40, -20],
            signal_sign = "positive"
            )

_ = prova_wfset.analyse(label=analysis_label ,analysis_class=analysis_class, input_parameters=ip, checks_kwargs = IPDict({'points_no': FS_wfset.points_per_wf}), overwrite=True)


In [ ]:
plotting_overlap_wf_PEAK_NEW(prova_wfset, n_wf= 2, show  = True, peak_bool = True, peak_beam_bool = True, analysis_label = analysis_label)

In [ ]:
# Selecting beam waveforms with just one peak 
def filter_beam_peaks_0(waveform: Waveform, analysis_label) -> bool:
    if len(waveform.analyses[analysis_label].result['beam_peak_index']) == 1:
        return True
    else:
        return False
    
prova_wfset = WaveformSet.from_filtered_WaveformSet(prova_wfset, filter_beam_peaks_0, analysis_label)


In [ ]:
# To cut full-streaming waveforms around the peak

# The waveform must have the beaseline around zero and the signal inverted (positive peaks)


from waffles.data_classes.WaveformAdcs import WaveformAdcs
from waffles.Exceptions import GenerateExceptionMessage
import numpy as np

def cut_full_streaming_window_and_inverting(
        waveform: WaveformAdcs,
        analysis_label: str,
        amplitude_threshold: float = 0.3,
        pre_peak_window: int = 50,
        window_timeticks_length: int = 588 #check the template, to have the same length
) -> None:
    """This method overwrites the adcs method of the given
    WaveformAdcs object.

    waveform: WaveformAdcs
        The waveform whose adcs will be modified
    analysis_label: str
        Name of the analysis including the peak information
    amplitude_threshold: float
        Percentage of the peak amplitude to define the starting point for the cut 
    """

    try:
        peak_index = waveform.analyses[analysis_label].result['beam_peak_index']
        baseline = waveform.analyses[analysis_label].result['mean_baseline']
    except KeyError:
        raise Exception(
            GenerateExceptionMessage(
                1,
                "Problem finding 'beam_peak_index' or 'beam_peak_amplitude' "
            )
        )
    
    # if len(peak_index) != 1:
    #     print("⚠️ peak_index length != 1, skipping…")
    #     return
    # else:
    peak_index = waveform.analyses[analysis_label].result['beam_peak_index'][0]
    peak_amplitude = waveform.analyses[analysis_label].result['beam_peak_amplitude'][0]

    # Searching for the index associated to peak_amplitude * amplitude_threshold (% of the raising edge)
    amplitude_threshold_dac = peak_amplitude * amplitude_threshold

    window = waveform.adcs[peak_index-pre_peak_window : peak_index]
    idx_in_window = np.abs(window - amplitude_threshold_dac).argmin()
    idx = (peak_index - pre_peak_window) + idx_in_window

    start = idx - pre_peak_window
    stop  = start + window_timeticks_length

    if start < 0:
        start = 0
        stop = window_timeticks_length

    if stop > len(waveform.adcs):
        stop = len(waveform.adcs)
        start = stop - window_timeticks_length

    # Ultimo controllo: la lunghezza DEVE essere esattamente quella
    if stop - start != window_timeticks_length:
        print("Skipping: cannot enforce fixed window length")

    # ---- PARTE 5: slicing garantito omogeneo
    waveform._Waveform__slice_adcs(start, stop)

    return 


In [ ]:

prova_wfset.apply(cut_full_streaming_window_and_inverting, analysis_label = analysis_label ,show_progress=False)

#new_pwak = prova_wfset.waveforms[0].analyses[analysis_label].result['beam_peak_index']-prova_wfset.waveforms[0].starting_tick

In [ ]:
plotting_overlap_wf_PEAK_NEW(prova_wfset, n_wf= 3, show  = True, peak_bool = False, peak_beam_bool = True, analysis_label = analysis_label)

In [ ]:
# Reading templates and createing a datagrame with three column: 'endpoint' 'channel' 'Template_avg' (i.e. adcs)


directory_path = '/afs/cern.ch/work/a/anbalbon/private/waffles/src/waffles/np04_analysis/lightyield_vs_energy/scripts/Renan_scripts/v_27xxx_fix_avg'
template_files = os.listdir(directory_path)

# Filter only files (not directories)
template_files = [f for f in template_files if os.path.isfile(os.path.join(directory_path, f))]

edp_list = []
ch_list  = []
adc_list = []

for f in template_files:
    try:
        print(f"Loading: {f}")
        parts = f.split()
        if len(parts) < 3 or "_" not in parts[2]:
            print(f"Unexpected filename format: {f}")
            continue

        edp = parts[2].split("_")[0]
        ch = parts[2].split("_")[1].split('.')[0]

        with open(os.path.join(directory_path, f), 'rb') as file:
            temp = pickle.load(file)

        adc_list.append(temp)
        edp_list.append(edp)
        ch_list.append(ch)

    except Exception as e:
        print(f"Error loading {f}: {e}")

df_template = pd.DataFrame({
    'endpoint' : edp_list,
    'channel' : ch_list,
    'Template_avg': adc_list})
df_template["endpoint"] = df_template["endpoint"].astype(int)
df_template["channel"]  = df_template["channel"].astype(int)



In [ ]:
# Selecting just one channel
apa = 1
end = 104
ch = 10

grid_apa_FS = ChannelWsGrid(APA_map[apa], prova_wfset, compute_calib_histo=False)
print(len(grid_apa_FS.ch_wf_sets[end][ch].waveforms[0].adcs))

# Looking at the template length --> for now i put the right umber 588 manually in the analysis for cutting the full-straming waveform, but this length must be put as input of the analysis (channel by channel)
template_adcs = df_template.loc[(df_template['endpoint'] == end) & (df_template['channel'] == ch), 'Template_avg'].iloc[0]
print(len(template_adcs))

In [ ]:
#trying to implement renan code

In [ ]:
import waffles.np04_analysis.lightyield_vs_energy.scripts.Renan_scripts.function as renan_function_module
from waffles.np04_analysis.lightyield_vs_energy.scripts.Renan_scripts.function import *


import waffles.np04_analysis.lightyield_vs_energy.scripts.Renan_scripts.photon_fix_v2  as renan_photon_fix_module
from waffles.np04_analysis.lightyield_vs_energy.scripts.Renan_scripts.photon_fix_v2 import photon_counter

In [ ]:
importlib.reload(renan_function_module) 
from waffles.np04_analysis.lightyield_vs_energy.scripts.Renan_scripts.function import *

importlib.reload(renan_photon_fix_module) 
from waffles.np04_analysis.lightyield_vs_energy.scripts.Renan_scripts.photon_fix_v2 import photon_counter

photon_counter(grid_apa_FS.ch_wf_sets[end][ch].waveforms[0].adcs, template_adcs, end, 256),

In [ ]:
# single waveform plot + template

A_s = 5.05010167e+00
A_t = 1.37561546e-01
tau_s = 7.44701889e+00
tau_t = 1.30849065e+03
offset = 1.76270252e-01
eA_s = 7.73087575e-01
eA_t = 2.96854065e-03
etau_s = 1.84452413e+00
etau_t =  4.55349461e+01
eoffset = 2.32843177e+00

fit = [np.float64(225.06377741694612), np.float64(350.23753695268294), np.float64(431.1505036236092), np.float64(491.0527437600059), np.float64(539.5383767651116), np.float64(580.4087697907571), np.float64(615.0952201524951), np.float64(644.3864600085757), np.float64(668.8942791958066), np.float64(689.1279949070931), np.float64(705.5245334428253), np.float64(718.4980570357968), np.float64(728.4797873277818), np.float64(735.9149291522167), np.float64(741.1688555729955), np.float64(744.4814128450972), np.float64(746.0288390864405), np.float64(745.9297307247292), np.float64(744.2704084781159), np.float64(741.119809032561), np.float64(736.5354086670474), np.float64(730.5809127469074), np.float64(723.3702638089521), np.float64(715.0531422465972), np.float64(705.7999032971145), np.float64(695.7955953672101), np.float64(685.235820484457), np.float64(674.332886773616), np.float64(663.3086218283654), np.float64(652.3220734661279), np.float64(641.4299676449334), np.float64(630.6437361653016), np.float64(619.9661213982516), np.float64(609.3990380581914), np.float64(598.9405242369452), np.float64(588.5829259486707), np.float64(578.3280699068162), np.float64(568.2393697184385), np.float64(558.423329211968), np.float64(548.9832656564887), np.float64(539.9957599147543), np.float64(531.5159674563379), np.float64(523.5470475950857), np.float64(516.0561429082159), np.float64(508.97470572832424), np.float64(502.19475124113865), np.float64(495.60108531875045), np.float64(489.1339150518363), np.float64(482.7747608326437), np.float64(476.5164514027773), np.float64(470.34901677264), np.float64(464.2641298172683), np.float64(458.2712604033553), np.float64(452.3924815643407), np.float64(446.6388707609945), np.float64(440.99645404852805), np.float64(435.4391151487145), np.float64(429.93555076698266), np.float64(424.45192908237334), np.float64(418.9498798247735), np.float64(413.38480317915423), np.float64(407.7206575270995), np.float64(401.98718838085904), np.float64(396.2623997962837), np.float64(390.6183501015264), np.float64(385.0901068470191), np.float64(379.6798910992756), np.float64(374.29570807318544), np.float64(368.7800203678395), np.float64(363.0481613149662), np.float64(357.17977033376627), np.float64(351.31107720399854), np.float64(345.47284129185493), np.float64(339.60724105902807), np.float64(333.6732992453773), np.float64(327.7108063436376), np.float64(321.7869903739006), np.float64(315.90953915719854), np.float64(310.0363424457316), np.float64(304.14278821946886), np.float64(298.2649632136991), np.float64(292.4565531779074), np.float64(286.7065231572263), np.float64(280.95075820061334), np.float64(275.1411940173267), np.float64(269.2909372809853), np.float64(263.4406517277743), np.float64(257.6261253145107), np.float64(251.87461185720463), np.float64(246.18648900872657), np.float64(240.51749307252555), np.float64(234.8119754322676), np.float64(229.0736060215295), np.float64(223.35521227641811), np.float64(217.7044756971707), np.float64(212.12865852319337), np.float64(206.61295014606947), np.float64(201.1223183341426), np.float64(195.6090768886099), np.float64(190.04056282671334), np.float64(184.4210086469527), np.float64(178.77699941626273), np.float64(173.1700982756504), np.float64(167.68637195355686), np.float64(162.37575187085494), np.float64(157.20191032649547), np.float64(152.09279426040956), np.float64(147.0094918431439), np.float64(141.9463160704844), np.float64(136.89713866851787), np.float64(131.83304278096236), np.float64(126.7175902655679), np.float64(121.54025581844097), np.float64(116.3137245414657), np.float64(111.07083237654527), np.float64(105.86899492247308), np.float64(100.76846692837252), np.float64(95.76621288271144), np.float64(90.80541609498624), np.float64(85.83875456967588), np.float64(80.87550749299199), np.float64(75.95119274308014), np.float64(71.09260641346718), np.float64(66.31431069995458), np.float64(61.62026287857438), np.float64(57.001071624632175), np.float64(52.43829361427138), np.float64(47.89234327636199), np.float64(43.30727063712354), np.float64(38.65484747082366), np.float64(33.977030745946315), np.float64(29.340763779837257), np.float64(24.74344969538893), np.float64(20.117277946136205), np.float64(15.407796948099419), np.float64(10.63720358891281), np.float64(5.866420256190777), np.float64(1.1633510106939573), np.float64(-3.405599196623503), np.float64(-7.812203097227321), np.float64(-12.110166871218448), np.float64(-16.38457008634792), np.float64(-20.651345258579262), np.float64(-24.858711921517163), np.float64(-28.96107996332374), np.float64(-32.978554443266184), np.float64(-36.9640096591866), np.float64(-40.96797819905965), np.float64(-45.03219270941656), np.float64(-49.171038876531696), np.float64(-53.344325722506724), np.float64(-57.49034250657649), np.float64(-61.5923121747206), np.float64(-65.67866454698148), np.float64(-69.76359696562668), np.float64(-73.79269123842492), np.float64(-77.68546439382783), np.float64(-81.43719320910101), np.float64(-85.11839275555653), np.float64(-88.78586361312864), np.float64(-92.4028878831885), np.float64(-95.89490556382674), np.float64(-99.27520525243024), np.float64(-102.6471299544235), np.float64(-106.10029533684313), np.float64(-109.61629802924496), np.float64(-113.12322853139993), np.float64(-116.5667779530624), np.float64(-119.92112426886911), np.float64(-123.2020069117888), np.float64(-126.50048184850378), np.float64(-129.93183736066865), np.float64(-133.48435092895056), np.float64(-137.0195902112796), np.float64(-140.4070113248372), np.float64(-143.6464642657879), np.float64(-146.8040899610265), np.float64(-149.92917573044718), np.float64(-153.03955059097518), np.float64(-156.1262213457197), np.float64(-159.14170009889713), np.float64(-162.02760898989044), np.float64(-164.80767520935558), np.float64(-167.58838405823684), np.float64(-170.45526233566787), np.float64(-173.36505196251332), np.float64(-176.2155895113132), np.float64(-178.99273032150856), np.float64(-181.78337541473803), np.float64(-184.67172178290105), np.float64(-187.64044137730295), np.float64(-190.61630914660486), np.float64(-193.51005066551707), np.float64(-196.22991464219197), np.float64(-198.7358777017459), np.float64(-201.12580307271853), np.float64(-203.55888633867005), np.float64(-206.08026658968345), np.float64(-208.60446025058306), np.float64(-211.04293722436884), np.float64(-213.4266555757818), np.float64(-215.84677410653126), np.float64(-218.34033688916762), np.float64(-220.87366329924478), np.float64(-223.3909031909537), np.float64(-225.84697186006343), np.float64(-228.2068430219509), np.float64(-230.47094376904255), np.float64(-232.67385552702416), np.float64(-234.84614487251096), np.float64(-236.97239937345452), np.float64(-239.0138503619377), np.float64(-240.95222763739187), np.float64(-242.79697146641857), np.float64(-244.57587424391824), np.float64(-246.33914051133974), np.float64(-248.14601964562232), np.float64(-250.02545185167673), np.float64(-251.97177161891358), np.float64(-253.9663115661372), np.float64(-255.99016843205683), np.float64(-258.0245797885834), np.float64(-260.0460685204329), np.float64(-262.02599741211924), np.float64(-263.94290254063975), np.float64(-265.8020663929486), np.float64(-267.6241821032774), np.float64(-269.43963896618), np.float64(-271.28507708713835), np.float64(-273.1733096194991), np.float64(-275.03947674129216), np.float64(-276.7836261684149), np.float64(-278.40368078599107), np.float64(-280.01550739654755), np.float64(-281.7286281153138), np.float64(-283.49793136131655), np.float64(-285.19460714358644), np.float64(-286.74624353850714), np.float64(-288.16774852729964), np.float64(-289.50576211402546), np.float64(-290.802032180363), np.float64(-292.09166259201703), np.float64(-293.38361725287854), np.float64(-294.65931903438775), np.float64(-295.8952894663643), np.float64(-297.0857130384478), np.float64(-298.2365325511083), np.float64(-299.3705680833438), np.float64(-300.5264052282363), np.float64(-301.73058685931284), np.float64(-302.95170385932136), np.float64(-304.1283820005471), np.float64(-305.24387692941673), np.float64(-306.3424436231503), np.float64(-307.47614377687097), np.float64(-308.6454692131573), np.float64(-309.8196353416694), np.float64(-310.9618605951086), np.float64(-312.03818306444475), np.float64(-313.0310511445777), np.float64(-313.9737850071412), np.float64(-314.92606304934486), np.float64(-315.90666361373917), np.float64(-316.8772597065472), np.float64(-317.7874771669456), np.float64(-318.62182300256393), np.float64(-319.3874728323366), np.float64(-320.10752169331533), np.float64(-320.81678968940207), np.float64(-321.5501532640207), np.float64(-322.32661871374745), np.float64(-323.15452251903247), np.float64(-324.0226764405249), np.float64(-324.89965552533636), np.float64(-325.7485421008347), np.float64(-326.54138908261336), np.float64(-327.25705747630565), np.float64(-327.8923952060223), np.float64(-328.4641263872859), np.float64(-328.99842805799824), np.float64(-329.5270417786085), np.float64(-330.08422566901487), np.float64(-330.6990921334604), np.float64(-331.3934699441952), np.float64(-332.1721949228078), np.float64(-332.98845665310205), np.float64(-333.76607410489765), np.float64(-334.45128026226985), np.float64(-335.0291403969483), np.float64(-335.50029553106776), np.float64(-335.8658249000561), np.float64(-336.1278909480927), np.float64(-336.32646453134424), np.float64(-336.5497181886348), np.float64(-336.8748041185351), np.float64(-337.2627270397583), np.float64(-337.6059563914967), np.float64(-337.8596216969667), np.float64(-338.0858001862614), np.float64(-338.36657310314325), np.float64(-338.6934798740722), np.float64(-339.0001343492266), np.float64(-339.23994065699145), np.float64(-339.41430530501094), np.float64(-339.5440174535894), np.float64(-339.64908261121604), np.float64(-339.7465331097117), np.float64(-339.8374178746962), np.float64(-339.90258090766076), np.float64(-339.92391369618645), np.float64(-339.92072486829744), np.float64(-339.93484078714266), np.float64(-339.97770770533174), np.float64(-340.0107677025444), np.float64(-339.98943059836967), np.float64(-339.92899582959967), np.float64(-339.8829822140449), np.float64(-339.8797730544681), np.float64(-339.89790640501167), np.float64(-339.89991576373643), np.float64(-339.8678575563403), np.float64(-339.79814290934064), np.float64(-339.69030556254074), np.float64(-339.54233869034437), np.float64(-339.35667815398955), np.float64(-339.15771129019964), np.float64(-338.9829804776697), np.float64(-338.84783691530953), np.float64(-338.7308434105692), np.float64(-338.6060426678131), np.float64(-338.4947327255825), np.float64(-338.4479763821804), np.float64(-338.4658404698279), np.float64(-338.462807351979), np.float64(-338.3448604670035), np.float64(-338.1407664684657), np.float64(-337.95994201045477), np.float64(-337.84712024343054), np.float64(-337.7182188353867), np.float64(-337.4654217298254), np.float64(-337.12091842585954), np.float64(-336.8120915086961), np.float64(-336.6107203196962), np.float64(-336.4646440600996), np.float64(-336.28201575581414), np.float64(-336.0315667842741), np.float64(-335.72770680810925), np.float64(-335.38808283975567), np.float64(-335.0145005320376), np.float64(-334.6054826097986), np.float64(-334.17671593507225), np.float64(-333.7557146846743), np.float64(-333.3577712208952), np.float64(-332.9742453643045), np.float64(-332.58814168805856), np.float64(-332.1916355179816), np.float64(-331.78386348414955), np.float64(-331.3546992878344), np.float64(-330.87655847608454), np.float64(-330.3196093137881), np.float64(-329.68048996908306), np.float64(-328.975526136503), np.float64(-328.2455242021), np.float64(-327.5598485173671), np.float64(-326.98921114087966), np.float64(-326.5483852497163), np.float64(-326.2108306317329), np.float64(-325.9029288396404), np.float64(-325.49715177713114), np.float64(-324.87579008479025), np.float64(-324.0926452751644), np.float64(-323.32155071826236), np.float64(-322.6815042901815), np.float64(-322.1437396403813), np.float64(-321.6220374619038), np.float64(-321.0690589812369), np.float64(-320.47338375903337), np.float64(-319.8493538006126), np.float64(-319.23480853733827), np.float64(-318.6702563073833), np.float64(-318.1561048406155), np.float64(-317.663216850115), np.float64(-317.16444016363613), np.float64(-316.6512783423986), np.float64(-316.12388701439727), np.float64(-315.5842432775843), np.float64(-315.034448697564), np.float64(-314.47387389972533), np.float64(-313.89726336267944), np.float64(-313.2967838507111), np.float64(-312.66173462269944), np.float64(-311.97978885030113), np.float64(-311.24640667523994), np.float64(-310.4711353295034), np.float64(-309.66961634656053), np.float64(-308.856363845504), np.float64(-308.04420431746377), np.float64(-307.2432086520334), np.float64(-306.45961133792986), np.float64(-305.69789177596687), np.float64(-304.96194373644454), np.float64(-304.25533697962413), np.float64(-303.57582776102157), np.float64(-302.91121165657114), np.float64(-302.23918459955325), np.float64(-301.50431354586783), np.float64(-300.62860126593114), np.float64(-299.60343582923406), np.float64(-298.5561391916709), np.float64(-297.6496879012875), np.float64(-296.88758176430036), np.float64(-296.14473340947274), np.float64(-295.3095545624841), np.float64(-294.37032074342625), np.float64(-293.3648923354157), np.float64(-292.352263380737), np.float64(-291.40015459866765), np.float64(-290.54955957705386), np.float64(-289.78730747667146), np.float64(-289.078359685172), np.float64(-288.40586154603744), np.float64(-287.7704512569358), np.float64(-287.16294201809495), np.float64(-286.5453070796927), np.float64(-285.87312354406976), np.float64(-285.14829886469687), np.float64(-284.41060419040923), np.float64(-283.6889853984331), np.float64(-282.9683573953989), np.float64(-282.2131057457273), np.float64(-281.3885821718282), np.float64(-280.46472188955266), np.float64(-279.44485062566673), np.float64(-278.3932301040728), np.float64(-277.3935815699427), np.float64(-276.4688640881795), np.float64(-275.5891039331517), np.float64(-274.71464592470676), np.float64(-273.82146480502195), np.float64(-272.89694019202375), np.float64(-271.9541389367317), np.float64(-271.02522255860015), np.float64(-270.1283225452902), np.float64(-269.24119428002206), np.float64(-268.3272794752641), np.float64(-267.3844620939869), np.float64(-266.4418468169655), np.float64(-265.5284317607423), np.float64(-264.6518060427095), np.float64(-263.805942925135), np.float64(-262.96138696383116), np.float64(-262.07137595852714), np.float64(-261.1012421341141), np.float64(-260.0530310418771), np.float64(-258.94732706493573), np.float64(-257.8119598822877), np.float64(-256.6776792141315), np.float64(-255.57304206205316), np.float64(-254.51986785767926), np.float64(-253.53140729875054), np.float64(-252.5796634904832), np.float64(-251.60288450609642), np.float64(-250.56581035477862), np.float64(-249.51329326819123), np.float64(-248.52101476999985), np.float64(-247.60174457622443), np.float64(-246.7077425928255), np.float64(-245.80165790554324), np.float64(-244.91290135581247), np.float64(-244.09315725579373), np.float64(-243.30883023047326), np.float64(-242.44692362144409), np.float64(-241.4157575941362), np.float64(-240.22991832187367), np.float64(-238.9577938385367), np.float64(-237.6777325790941), np.float64(-236.466402922634), np.float64(-235.37618080780598), np.float64(-234.40679576262067), np.float64(-233.532994929793), np.float64(-232.73456094059568), np.float64(-232.00066673020856), np.float64(-231.3112062689486), np.float64(-230.61805179949954), np.float64(-229.86271434390176), np.float64(-229.01878795866554), np.float64(-228.09131148663403), np.float64(-227.10082480460412), np.float64(-226.0780188923455), np.float64(-225.05407148293295), np.float64(-224.02352983376576), np.float64(-222.94710528375097), np.float64(-221.79395008804497), np.float64(-220.5805637868269), np.float64(-219.34912597040903), np.float64(-218.15584596862237), np.float64(-217.0644967401265), np.float64(-216.11209251639792), np.float64(-215.26597664384363), np.float64(-214.4586122466982), np.float64(-213.6242276959376), np.float64(-212.7064207498239), np.float64(-211.68670985179637), np.float64(-210.62974466349613), np.float64(-209.6315218347815), np.float64(-208.6892305384539), np.float64(-207.6988767571615), np.float64(-206.58463544167898), np.float64(-205.4279552639915), np.float64(-204.3794699355317), np.float64(-203.4705696517872), np.float64(-202.60169023127403), np.float64(-201.66944870987177), np.float64(-200.67819384643337), np.float64(-199.68446849370673), np.float64(-198.6965481745442), np.float64(-197.66341991165217), np.float64(-196.54403581270302), np.float64(-195.37675174456533), np.float64(-194.23822791880335), np.float64(-193.16845456192632), np.float64(-192.16208867759573), np.float64(-191.20244261809245), np.float64(-190.28593353959906), np.float64(-189.41383379301578), np.float64(-188.55941485163993), np.float64(-187.666714377867), np.float64(-186.69248016457766), np.float64(-185.65484039307495), np.float64(-184.60304276025232), np.float64(-183.57157455988158), np.float64(-182.5723161497236), np.float64(-181.6096183176787), np.float64(-180.68972666135812), np.float64(-179.8193999232302), np.float64(-178.99496770814773), np.float64(-178.20188591327027), np.float64(-177.4166345018872), np.float64(-176.60240368043554), np.float64(-175.71878786663487), np.float64(-174.76290363123508), np.float64(-173.77171535380833), np.float64(-172.7830157542464), np.float64(-171.79608570518639), np.float64(-170.78908626486168), np.float64(-169.74158202444792), np.float64(-168.6401974468782), np.float64(-167.48875922221595), np.float64(-166.33195685769635), np.float64(-165.23332982182967), np.float64(-164.2208325365034), np.float64(-163.27951537998968), np.float64(-162.36833736730992), np.float64(-161.41769140691915), np.float64(-160.3494243220567), np.float64(-159.16004064101898), np.float64(-157.92871725481177), np.float64(-156.7414865276209), np.float64(-155.61362218299212), np.float64(-154.52209512149375), np.float64(-153.46602070815524), np.float64(-152.47938561538294), np.float64(-151.5941747479164), np.float64(-150.79719011251532), np.float64(-150.05054756138557), np.float64(-149.32718013695228), np.float64(-148.61895597316845), np.float64(-147.9199566837906), np.float64(-147.20942281190935), np.float64(-146.45909755258677), np.float64(-145.65681259187426), np.float64(-144.81052573602184), np.float64(-143.93091944392236), np.float64(-143.01348861032494), np.float64(-142.04690793160034), np.float64(-141.04794713877075), np.float64(-140.06719551023696), np.float64(-139.149856723801), np.float64(-138.28337082445842), np.float64(-137.42339077500264), np.float64(-136.54406671329542), np.float64(-135.65033939185432), np.float64(-134.74831692418678), np.float64(-133.81007499381778), np.float64(-132.7917713021847), np.float64(-131.70833803853458), np.float64(-130.64778726379222), np.float64(-129.69345129657543), np.float64(-128.82102472797777), np.float64(-127.94618273411315), np.float64(-127.02845608332532), np.float64(-126.09779908666376), np.float64(-125.19971319000065), np.float64(-124.3402862385822), np.float64(-123.50070295995674), np.float64(-122.65538555314767), np.float64(-121.77873289611196), np.float64(-120.85509927868742), np.float64(-119.90257369953814), np.float64(-118.95740688335398), np.float64(-118.0356282260715), np.float64(-117.12296409481183), np.float64(-116.20406239462778), np.float64(-115.30080554803823), np.float64(-114.45517540012807), np.float64(-113.66969716847376), np.float64(-112.89202777146939), np.float64(-112.06577430775495), np.float64(-111.19665513309947), np.float64(-110.32698416698896), np.float64(-109.4763880966349), np.float64(-108.62394026846809), np.float64(-107.74252817051388), np.float64(-106.84064960268834), np.float64(-105.94787506490985), np.float64(-105.07313821790875), np.float64(-104.19218679054936), np.float64(-103.27523589810983), np.float64(-102.32078355569486), np.float64(-101.3467263134932), np.float64(-100.3973536789634), np.float64(-99.54352908188741), np.float64(-98.83006265880093), np.float64(-98.16493870137575), np.float64(-97.37888837687937), np.float64(-96.40936160165548), np.float64(-95.35892581378926), np.float64(-94.371413261033), np.float64(-93.49435049130344), np.float64(-92.71016533827991), np.float64(-91.9735560973902), np.float64(-91.22728250033695), np.float64(-90.42298292851775), np.float64(-89.5652724973229), np.float64(-88.69132564108776), np.float64(-87.83645749403749), np.float64(-87.02122433405056), np.float64(-86.254887077882), np.float64(-85.52574392873073), np.float64(-84.80996413883003), np.float64(-84.08849941054852), np.float64(-83.353364722368), np.float64(-82.60205826052407), np.float64(-81.83607079388905), np.float64(-81.0596692763795), np.float64(-80.2916198784985), np.float64(-79.56923759845897), np.float64(-78.93181378969474), np.float64(-78.3950506598837), np.float64(-77.95669795614513), np.float64(-77.56066500312774), np.float64(-77.08416645431201), np.float64(-76.40932223441298), np.float64(-75.55517090314841), np.float64(-74.6287459456659), np.float64(-73.71520907868407), np.float64(-72.83324946451329), np.float64(-71.98348266157215), np.float64(-71.20723596863014), np.float64(-70.57157876736818), np.float64(-70.07397310105024), np.float64(-69.60297089451804), np.float64(-69.02348562049235), np.float64(-68.29656232253193), np.float64(-67.450790101581), np.float64(-66.54872277776586), np.float64(-65.6726639357216), np.float64(-64.89418308730214), np.float64(-64.20024442819445), np.float64(-63.522898616839946), np.float64(-62.81181157342235), np.float64(-62.06606493808022), np.float64(-61.303272558645034), np.float64(-60.53032121507441), np.float64(-59.74509176811498), np.float64(-58.95062881063105), np.float64(-58.16155603848894), np.float64(-57.39440723635554), np.float64(-56.65149148825578), np.float64(-55.925682201952675), np.float64(-55.221940324668175), np.float64(-54.567500582575825), np.float64(-53.987944162206006), np.float64(-53.45578917887695), np.float64(-52.907724800995226), np.float64(-52.299039851254435), np.float64(-51.629683593415955), np.float64(-50.91781211504782), np.float64(-50.17712189730376), np.float64(-49.4166349622075), np.float64(-48.657819439552185), np.float64(-47.94339260030712), np.float64(-47.31263605416214), np.float64(-46.74212448230928), np.float64(-46.16520230368449), np.float64(-45.529964629136195), np.float64(-44.82822688565313), np.float64(-44.07377453270682), np.float64(-43.295596420851005), np.float64(-42.53100394841261), np.float64(-41.81864832924884), np.float64(-41.1951698744303), np.float64(-40.68683236667363), np.float64(-40.270388399381936), np.float64(-39.88782819309227), np.float64(-39.466106425484185), np.float64(-38.92558825356563), np.float64(-38.20672919307005), np.float64(-37.37113373674359), np.float64(-36.56446464471932), np.float64(-35.891534040236216), np.float64(-35.347918976290444), np.float64(-34.88296627484578), np.float64(-34.45605152901446), np.float64(-34.03994399037595), np.float64(-33.61755472181955), np.float64(-33.181572592766926), np.float64(-32.727522904140635), np.float64(-32.2438745160668), np.float64(-31.713758220010746), np.float64(-31.13073707424188), np.float64(-30.50863824288842), np.float64(-29.867477261572173), np.float64(-29.211457735295685), np.float64(-28.5325854932883), np.float64(-27.83685692303656), np.float64(-27.160555512291513), np.float64(-26.545572552733915), np.float64(-25.987857277388112), np.float64(-25.44841303763991), np.float64(-24.90320700815462), np.float64(-24.373261894490856), np.float64(-23.889382768302188), np.float64(-23.420615802080928), np.float64(-22.888713853245783), np.float64(-22.261521317057674), np.float64(-21.614026468878006), np.float64(-21.05268269634514), np.float64(-20.581828133527278), np.float64(-20.123347903415333), np.float64(-19.617449332835307), np.float64(-19.084265755783708), np.float64(-18.575590770753326), np.float64(-18.112556755213426), np.float64(-17.687682073630207), np.float64(-17.282724820397863), np.float64(-16.876882045903947), np.float64(-16.449359142457013), np.float64(-15.985709085929788), np.float64(-15.476672731810945), np.float64(-14.926239004939237), np.float64(-14.359485152899534), np.float64(-13.811427328506696), np.float64(-13.318784366064246), np.float64(-12.917471363185875), np.float64(-12.605645082102779), np.float64(-12.311931461789905), np.float64(-11.944040747198638), np.float64(-11.482287251227262), np.float64(-10.968291904510727), np.float64(-10.444383646291131), np.float64(-9.91595857104877), np.float64(-9.37213632105208), np.float64(-8.810350806157706), np.float64(-8.237194024813938), np.float64(-7.648856630663016), np.float64(-7.015838780995505), np.float64(-6.307922858329248), np.float64(-5.571945245861013), np.float64(-4.91752951991754), np.float64(-4.4260721611918346), np.float64(-4.083146564467871), np.float64(-3.8276042281476803), np.float64(-3.5880760332083144), np.float64(-3.292929131153521), np.float64(-2.9050267074704097), np.float64(-2.4556341027813406), np.float64(-2.0045520122745177), np.float64(-1.586331448937014), np.float64(-1.2093393245303015), np.float64(-0.8621082073948609), np.float64(-0.5111325766861898), np.float64(-0.11724679031956514), np.float64(0.3256672370099081), np.float64(0.7948279721327103), np.float64(1.264957837729087), np.float64(1.7257200979919078), np.float64(2.1730372748256457), np.float64(2.5948552947118264), np.float64(2.9713892367884096), np.float64(3.3010224842098603), np.float64(3.6251719817132155), np.float64(4.000173578275449), np.float64(4.4380824746801055), np.float64(4.909828452101858), np.float64(5.369995980670294), np.float64(5.768571203909898), np.float64(6.06019864495359), np.float64(6.250677878083744), np.float64(6.389784135265562), np.float64(6.544084216915754), np.float64(6.783415936487951), np.float64(7.169410886855921), np.float64(7.687756439621246), np.float64(8.258640330890096), np.float64(8.798227173961154), np.float64(9.263457656657675), np.float64(9.635451850632784), np.float64(9.926240178609158), np.float64(10.170820019876363), np.float64(10.415166067559094), np.float64(10.711657155315894), np.float64(11.105595425928353), np.float64(11.558036785588243), np.float64(11.95567213966862), np.float64(12.21798369274645), np.float64(12.396596094071727), np.float64(12.601277498885656), np.float64(12.874554672173176), np.float64(13.185634470726102), np.float64(13.485656768440819), np.float64(13.743365632504483), np.float64(13.940237364423536), np.float64(14.08492304770808), np.float64(14.208610393007367), np.float64(14.354396240830864), np.float64(14.57435467934368), np.float64(14.917557930870412), np.float64(15.367306871091442), np.float64(15.84600869552975), np.float64(16.26542638746206), np.float64(16.56353332534019), np.float64(16.702124620094416), np.float64(16.735134174974487), np.float64(16.79830685646268), np.float64(17.008903798324155), np.float64(17.369410215446088), np.float64(17.82154276951982), np.float64(18.290216815769792), np.float64(18.697854891739887), np.float64(18.995925025835298), np.float64(19.209264672309192), np.float64(19.399922082141185), np.float64(19.629492361755265), np.float64(19.95052707223248), np.float64(20.369438805319977), np.float64(20.78612243106975), np.float64(21.05834061754165), np.float64(21.1657416529559), np.float64(21.216957068395242), np.float64(21.3310756518133), np.float64(21.535472091659198), np.float64(21.808275457733863), np.float64(22.12657754437311), np.float64(22.47813602825149), np.float64(22.854373347122493), np.float64(23.24622538079769), np.float64(23.644077478378517), np.float64(24.03793785169247), np.float64(24.417541324122478), np.float64(24.773700327756654), np.float64(25.100394422679226), np.float64(25.39204577839686), np.float64(25.63075349035862), np.float64(25.78606543840632), np.float64(25.84745386693502), np.float64(25.86968253013743), np.float64(25.93712568700705), np.float64(26.07937638303617), np.float64(26.262416574013766), np.float64(26.441733669270622), np.float64(26.60795001652255), np.float64(26.772181559245425), np.float64(26.953942871898715), np.float64(27.176147061061197), np.float64(27.445929560491003), np.float64(27.7234616913312), np.float64(27.947339036697368), np.float64(28.097093029035907), np.float64(28.200903550906627), np.float64(28.294853587829216), np.float64(28.38632641639187), np.float64(28.46835970034955), np.float64(28.557067838610426), np.float64(28.696978079060177), np.float64(28.923175644892282), np.float64(29.213176069544044), np.float64(29.512737877537063), np.float64(29.766949346617007), np.float64(29.928862253916407), np.float64(29.984967708934217), np.float64(30.01050731634641), np.float64(30.123098988395213), np.float64(30.367983937707248), np.float64(30.69973410348213), np.float64(31.045267908546545), np.float64(31.351200850836133), np.float64(31.58052815587055), np.float64(31.744060107358568), np.float64(31.900632777914556), np.float64(32.101761618605), np.float64(32.32175082554212), np.float64(32.49506352667962), np.float64(32.606286754129826), np.float64(32.70758279144687), np.float64(32.85617959051043), np.float64(33.04540772556055), np.float64(33.23153402495767), np.float64(33.371192611469716), np.float64(33.43243117820951), np.float64(33.40493811154332), np.float64(33.33258382094445), np.float64(33.28656549556156), np.float64(33.299555911882855), np.float64(33.35187698572647), np.float64(33.41020754858647), np.float64(33.46201832418115), np.float64(33.50865573576709), np.float64(33.56368084055498), np.float64(33.65051921806948), np.float64(33.78823981473879), np.float64(33.96926049918865), np.float64(34.17014407849301), np.float64(34.3607658684707), np.float64(34.5081373966393), np.float64(34.598424152172555), np.float64(34.685073796846936), np.float64(34.85441418213354), np.float64(35.10556060341751), np.float64(35.323340211570404), np.float64(35.38564679213719), np.float64(35.298436542629716), np.float64(35.14384939495178), np.float64(34.992055884776434), np.float64(34.873798382083464), np.float64(34.79910449384425), np.float64(34.75985051152912), np.float64(34.739534558952926), np.float64(34.73702767402185), np.float64(34.77276257431566), np.float64(34.87190979001693), np.float64(35.04624733225218), np.float64(35.297192104780436), np.float64(35.5947513238247), np.float64(35.87365681726924), np.float64(36.082917918317925), np.float64(36.273012271625845), np.float64(36.55052263196001), np.float64(36.93823553107998), np.float64(37.33448195437347), np.float64(37.61640799689637), np.float64(37.76559667247488), np.float64(37.828949852903186), np.float64(37.84235419142585), np.float64(37.804453420296944), np.float64(37.71083883331229), np.float64(37.60144080995483), np.float64(37.54297709689456), np.float64(37.580120781608514), np.float64(37.71952474390308), np.float64(37.93968395548092), np.float64(38.170281219278536), np.float64(38.316091612978425), np.float64(38.34793970567487), np.float64(38.331814813569395), np.float64(38.34968947874879), np.float64(38.39622493975897), np.float64(38.41180942237474), np.float64(38.37988029507687), np.float64(38.36118338791107), np.float64(38.4210950112362), np.float64(38.51853119791537), np.float64(38.547108841939405), np.float64(38.46109874716947), np.float64(38.321012992862045), np.float64(38.22002052878034), np.float64(38.20726227907623), np.float64(38.29921550924877), np.float64(38.47926963128214), np.float64(38.698875362232855), np.float64(38.90234965958738), np.float64(39.05858166322476), np.float64(39.153543895083416), np.float64(39.18746153533517), np.float64(39.17325329236951), np.float64(39.12970510834357), np.float64(39.07841197251186), np.float64(39.042265955057786), np.float64(39.04219452084645), np.float64(39.0958656519366), np.float64(39.207280658526685), np.float64(39.327530113952385), np.float64(39.37609932371432), np.float64(39.32033317866521), np.float64(39.20826973712731), np.float64(39.1102075856661), np.float64(39.046427938195414), np.float64(39.00113226844722), np.float64(38.94284693105032), np.float64(38.833075769406726)]


i=0
template_factor = 100

fig = go.Figure()

y = prova_wfset.waveforms[i].adcs 
x = np.arange(len(y))
    
fig.add_trace(go.Scatter(
    x=np.arange(len(prova_wfset.waveforms[i].adcs)),
    y=prova_wfset.waveforms[i].adcs ,
    mode='lines',
    line=dict(width=0.5),
    name = 'Cutted original waveform',
    showlegend=True))


fig.add_trace(go.Scatter(
    x=prova_wfset.waveforms[i].analyses[analysis_label].result['beam_peak_time']-prova_wfset.waveforms[i].starting_tick,
    y=prova_wfset.waveforms[i].analyses[analysis_label].result['beam_peak_amplitude'],
    mode='markers',
    marker=dict(color='blue', size=8, symbol='circle'),
    name='Beam peaks'
    ))

fig.add_trace(go.Scatter(
    x= np.arange(len(template_adcs)),
    y=template_adcs*template_factor,
    mode='lines',
    line=dict(width=0.5),
    name = f'Template waveform x*{template_factor}',
    showlegend=True))

fig.add_trace(go.Scatter(
    x= np.arange(len(fit)),
    y=fit,
    mode='lines',
    line=dict(width=0.5),
    name = f'Fitted fuction',
    showlegend=True))



fig.show()

In [ ]:
print("\nFS_wfset waveformset:")
print("points_per_wf:", FS_wfset.points_per_wf)
print("runs:", FS_wfset.runs)
print("record_numbers:", FS_wfset.record_numbers)
print("available_channels:", FS_wfset.available_channels)
print("mean_adcs:", FS_wfset.mean_adcs)
print("mean_adcs_idcs:", FS_wfset.mean_adcs_idcs)

print("\nprova_wfset waveformset:")
print("points_per_wf:", prova_wfset.points_per_wf)
print("runs:", prova_wfset.runs)
print("record_numbers:", prova_wfset.record_numbers)
print("available_channels:", prova_wfset.available_channels)
print("mean_adcs:", prova_wfset.mean_adcs)
print("mean_adcs_idcs:", prova_wfset.mean_adcs_idcs)

print("\n --------------------------------------------- \n")
i=2
print("\nprova_wfset waveform:")
print("timestamp:", prova_wfset.waveforms[i].timestamp)
print("daq_window_timestamp:", prova_wfset.waveforms[i].daq_window_timestamp)
print("run_number:", prova_wfset.waveforms[i].run_number)
print("record_number:", prova_wfset.waveforms[i].record_number)
print("endpoint:", prova_wfset.waveforms[i].endpoint)
print("channel:", prova_wfset.waveforms[i].channel)
print("starting_tick:", prova_wfset.waveforms[i].starting_tick)
print("trigger_type:", prova_wfset.waveforms[i].trigger_type)

print("\n\FS_wfset waveform:")
print("timestamp:", FS_wfset.waveforms[i].timestamp)
print("daq_window_timestamp:", FS_wfset.waveforms[i].daq_window_timestamp)
print("run_number:", FS_wfset.waveforms[i].run_number)
print("record_number:", FS_wfset.waveforms[i].record_number)
print("endpoint:", FS_wfset.waveforms[i].endpoint)
print("channel:", FS_wfset.waveforms[i].channel)
print("starting_tick:", FS_wfset.waveforms[i].starting_tick)
print("trigger_type:", FS_wfset.waveforms[i].trigger_type)